Ideas:

* Add FFT channels to CNN (Tried, does not make much difference)
* Finetune CNN (with SGD slow learning rate)
* 5-fold CNN
* Extract Features from CNN (before FC) and do XGB
* TTA (tried, made it better)
* More augmenting, additional 45, 135, 315 degrees
* More augmenting, random rotations and flips
* Predict test data and train with test
* Train on all of the training data (no train-val split)
* Try a different combination of combine predictions
* Fine-tune on pre-trained models (Get rid of some top layers because input size is small)

In [1]:
import numpy as np
import pandas as pd
from skimage import transform
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras import losses, optimizers, callbacks
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, train_test_split, StratifiedKFold, KFold
from scipy import fftpack
import random
import matplotlib.pyplot as plt
%matplotlib inline
RANDOM_SEED = 43
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

Using TensorFlow backend.
/home/can/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Create Dataset 

In [2]:
# Train
df = pd.read_json('./data/train.json')
X, y = [], []
for im_band1, im_band2, label in zip(df['band_1'], df['band_2'], df['is_iceberg']):
    im_band1 = np.array(im_band1).reshape(75, 75, 1)
    im_band2 = np.array(im_band2).reshape(75, 75, 1)    
    # Preprocess
    # - Zero mean
    im_band1 -= np.mean(im_band1)
    im_band2 -= np.mean(im_band2)
    # - Normalize
    im_band1 /= np.std(im_band1)
    im_band2 /= np.std(im_band2)    
    im = np.concatenate([im_band1, im_band2], axis=2)
    X.append(im)
    y.append(label)    
X = np.array(X)
y = np.array(y)
print 'X.shape:', X.shape
print 'y.shape:', y.shape

X.shape: (1604, 75, 75, 2)
y.shape: (1604,)


## Train - Val Split

In [3]:
N_SPLITS = 5
MODEL_NUMBER = 5
skf = StratifiedKFold(n_splits=N_SPLITS, random_state=RANDOM_SEED, shuffle=True)
cv = list(skf.split(X, y))

In [4]:
train_i, val_i = cv[MODEL_NUMBER - 1]
X_train, y_train = X[train_i], y[train_i]
X_val, y_val = X[val_i], y[val_i]
print 'X_train.shape:', X_train.shape
print 'y_train.shape:', y_train.shape
print 'X_val.shape:', X_val.shape
print 'y_val.shape:', y_val.shape
print 'np.mean(y_train):', np.mean(y_train)
print 'np.mean(y_val):', np.mean(y_val)

X_train.shape: (1284, 75, 75, 2)
y_train.shape: (1284,)
X_val.shape: (320, 75, 75, 2)
y_val.shape: (320,)
np.mean(y_train): 0.469626168224
np.mean(y_val): 0.46875


## Data Augmentation

In [5]:
def bypass(im):
    return im

def h_flip(im):
    return im[:, ::-1]

def v_flip(x):
    return im[::-1, :]

def hv_flip(x):
    return h_flip(v_flip(x))

def random_rot(im):
    angle = int((np.random.randint(3) + 1) * 90)
    return transform.rotate(im, angle=angle, mode='reflect')

aug_funcs = [h_flip, v_flip, random_rot]             

In [6]:
def data_generator(X, y, aug_funcs, batch_size=32, augment=True):
    """
    Generates random data
    """
    X = list(np.copy(X))
    y = list(np.copy(y))
    X_y = zip(X, y)
    while True:
        X_y_batch = random.sample(X_y, batch_size)
        X_batch = [e[0] for e in X_y_batch]
        y_batch = [e[1] for e in X_y_batch]
        # Random augmentation        
        for i, x in enumerate(X_batch):            
            funcs2apply = random.sample(aug_funcs, np.random.randint(len(aug_funcs) + 1))
            for f in funcs2apply:
                x = f(x)
            X_batch[i] = x
        yield np.array(X_batch), np.array(y_batch) 

In [13]:
# gen_obj = data_generator(X_train, y_train, batch_size=32, aug_funcs=aug_funcs)
next(gen_obj)

(array([[[[  1.64646565e+00,  -1.60878319e+00],
          [  1.09838933e+00,  -9.45093800e-01],
          [  1.26320902e+00,  -1.15458792e+00],
          ..., 
          [  2.51328846e-01,  -1.37666199e+00],
          [  7.44300160e-01,  -7.47149017e-01],
          [  1.57160931e+00,  -5.57186834e-01]],
 
         [[  1.18172089e+00,  -2.00497486e-01],
          [  1.34303309e+00,   8.44682883e-01],
          [  9.26038842e-01,   8.44664076e-01],
          ..., 
          [  7.44317180e-01,  -7.47131151e-01],
          [ -1.28567083e+00,  -1.61010954e+00],
          [  5.54709908e-01,  -7.47167824e-01]],
 
         [[  1.42121101e+00,   8.44700750e-01],
          [  8.36859743e-01,   1.45535178e+00],
          [  1.18168641e+00,   1.22190516e+00],
          ..., 
          [  2.51312262e-01,  -5.57168027e-01],
          [  2.51294806e-01,  -7.47167824e-01],
          [  6.50663938e-01,   1.25022673e-01]],
 
         ..., 
         [[ -1.28461034e+00,  -9.45260241e-01],
          [ -1.7

# Training

In [7]:
def get_model(input_shape):
    # Architecture
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    loss = losses.binary_crossentropy
    optimizer = optimizers.Adam()
    metrics = ['accuracy']
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    #     
    return model

In [8]:
model = get_model(input_shape=(75, 75, 2))

In [9]:
# Callbacks
def get_lr(epoch):
    lr = (np.random.rand() * 1e-3 + 1e-7)
    print 'lr:', lr
    return lr
MODEL_PATH = './models/model6/model' + str(MODEL_NUMBER) + '.h5'
m_q = 'val_loss'
model_path = MODEL_PATH
check_pt = callbacks.ModelCheckpoint(filepath=model_path, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=7, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor=m_q, verbose=1)
schedule_lr = callbacks.LearningRateScheduler(get_lr)
callback_list = [check_pt, early_stop, reduce_lr]

# Fit generator
batch_size = 32
steps_per_epoch = (len(X_train) / batch_size) * 8
gen_obj = data_generator(X_train, y_train, batch_size=batch_size, aug_funcs=aug_funcs)
model.fit_generator(gen_obj, steps_per_epoch=steps_per_epoch, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Epoch 1/200
320/320 [==============================] - 6s - loss: 0.6487 - acc: 0.5755 - val_loss: 0.6046 - val_acc: 0.6219
Epoch 2/200
320/320 [==============================] - 5s - loss: 0.5593 - acc: 0.6594 - val_loss: 0.3209 - val_acc: 0.8656
Epoch 3/200
320/320 [==============================] - 5s - loss: 0.5003 - acc: 0.6842 - val_loss: 0.3188 - val_acc: 0.8531
Epoch 4/200
320/320 [==============================] - 5s - loss: 0.4891 - acc: 0.6964 - val_loss: 0.2740 - val_acc: 0.8906
Epoch 5/200
320/320 [==============================] - 5s - loss: 0.4636 - acc: 0.7087 - val_loss: 0.2552 - val_acc: 0.8906
Epoch 6/200
320/320 [==============================] - 5s - loss: 0.4488 - acc: 0.7175 - val_loss: 0.2680 - val_acc: 0.8969
Epoch 7/200
320/320 [==============================] - 5s - loss: 0.4390 - acc: 0.7262 - val_loss: 0.2523 - val_acc: 0.8875
Epoch 8/200
320/320 [==============================] - 5s - loss: 0.4441 - acc: 0.7252 - val_loss: 0.3016 - val_acc: 0.8844
Epoch 9/

In [12]:
# # Finetune
# loss = losses.binary_crossentropy
# optimizer = optimizers.SGD(lr=1e-4)
# metrics = ['accuracy']
# model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [13]:
# m_q = 'val_loss'
# model_path = MODEL_PATH
# check_pt = callbacks.ModelCheckpoint(filepath=model_path, monitor=m_q, save_best_only=True, verbose=1)
# early_stop = callbacks.EarlyStopping(patience=5, monitor=m_q, verbose=1)
# reduce_lr = callbacks.ReduceLROnPlateau(patience=2, factor=0.33, monitor=m_q, verbose=1)
# callback_list = [check_pt, early_stop, reduce_lr]

# model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

# Predict Test

In [12]:
# Load test data
df = pd.read_json('./data/test.json')
X_test, y_test = [], []
for im_band1, im_band2 in zip(df['band_1'], df['band_2']):
    im_band1 = np.array(im_band1).reshape(75, 75, 1)
    im_band2 = np.array(im_band2).reshape(75, 75, 1)    
    # Preprocess - zero mean
    im_band1 -= np.mean(im_band1)
    im_band2 -= np.mean(im_band2)
    # Preprocess - normalize
    im_band1 /= np.std(im_band1)
    im_band2 /= np.std(im_band2)    
    im = np.concatenate([im_band1, im_band2], axis=2)
    X_test.append(im)    
X_test = np.array(X_test)
print 'X_test.shape:', X_test.shape

X_test.shape: (8424, 75, 75, 2)


In [13]:
# predict - tta
def bypass(x):
    return x

def h_flip(x):
    return x[:, :, ::-1, :]

def v_flip(x):
    return x[:, ::-1, :, :]

def hv_flip(x):
    return h_flip(v_flip(x))

def rot90(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 90), axis=0) for im in x], axis=0)

def rot180(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 180), axis=0) for im in x], axis=0)

def rot270(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 270), axis=0) for im in x], axis=0)

tta_aug_funcs = [bypass, 
                 h_flip, v_flip, hv_flip,
                 rot90, rot180, rot270]

In [14]:
y_test_p = 0
weights = [0.25, 0.5 / 3, 0.25, 0.5 / 3, 0.5 / 3]
for i, w in zip(range(5), weights):
    print i
    # Load the model
    MODEL_PATH = './models/model6/model' + str(i + 1) + '.h5'
    model = load_model(MODEL_PATH)
    # predict - tta    
    for func in tta_aug_funcs:
        y_test_p += model.predict(func(X_test), verbose=1).flatten() * w
# y_test_p = y_test_p / (len(aug_funcs) * 5.0)
y_test_p = y_test_p / (len(tta_aug_funcs) * sum(weights))

0
8424/8424 [==============================] - 1s     
4
8384/8424 [============================>.] - ETA: 0s

In [15]:
df_sub = pd.DataFrame()
df_sub['id'] = df['id']
df_sub['is_iceberg'] = y_test_p.flatten()
df_sub.to_csv('./submissions/sub16.csv', index=False)